# LangChain

 - 텍스트에서 SQL을 뽑아서 데이터베이스에 실행하고 결과를 가져온다. 
 - 외부의 Webhook, API 를 호출 
 - 여러 모델에서 결과를 가져오거나 모델을 연결해서 순서대로 실행
 
이런 시나리오가 필요할 때 LangChain. Components, Agents 
[Text](https://python.langchain.com/docs/integrations/llms/google_vertex_ai_palm), [Chat](https://python.langchain.com/docs/integrations/chat/google_vertex_ai_palm), and [Embeddings](https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm) 모델이 LangChain Python SDK와 통합됨. 

In [6]:
!pip install google-cloud-aiplatform chromadb pydantic typing-inspect

In [7]:
!pip install langchain

In [8]:
import time
from typing import List 

import langchain
from pydantic import BaseModel 

print(f"LangChain Version: {langchain.__version__}")

LangChain Version: 0.0.286


In [11]:
# vertext AI 
from google.cloud import aiplatform
from langchain.chat_models.vertexai import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.schema import HumanMessage, SystemMessage

print(f"Vertext AI SDK version: {aiplatform.__version__}")

Vertext AI SDK version: 1.32.0


In [ ]:
# Utility functions for Embeddings API with rate limiting 
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print('waiting')
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elabsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)
            
class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel): 
    requests_per_minute: int
    num_instances_per_batch: int 
    
    # overriding embed_documents method 
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)
        
        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)
            
        return [r.values for r in results]